# DART 기업 재무제표 수집해보기

In [1]:
### 재무제표 수집에 앞서 필요한 모듈들을 불러오자

from urllib.request import urlopen
from io import BytesIO
from zipfile import ZipFile
import xml.etree.ElementTree as ET
import pandas as pd

from time import sleep

import requests
import json
import os


from datetime import datetime

개인이 부여 받은 DART api 키 정보를 입력해준다

In [2]:
api_key = "개인이 부여받은 DART api_key"

read_csv를 불러와 데이터프레임으로 읽어보자. dtype처리로 숫자를 문자열로 변환하여 담아주자.

In [3]:
corp_info_stock_code = pd.read_csv("corp_info_stock_code.csv", dtype = {'corp_code':str, 'stock_code':str})

각 항목마다 val 변수를 만들어 값을 담아둔다.

In [1]:
corp_code_val = '00483735'
bsns_year_val = '2022'
reprt_code_val = '11011'
fs_div_val = 'CFS'

url 변수에 DART 재무제표 api 주소를 담고, 파라미터 변수를 만들어 f스트링을 활용하여 api_key와 위의 val 변수들을 담아주자.

In [5]:
url = 'https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json?'

params = f'crtfc_key={api_key}&corp_code={corp_code_val}&bsns_year={bsns_year_val}&reprt_code={reprt_code_val}&fs_div={fs_div_val}'

위의 두 변수를 간편하게 하나의 변수로 합쳐주자

In [6]:
api_url = url + params

리퀘스트 라이브러리로 위 주소의 원격 ip를 호출하자

In [7]:
r = requests.get(api_url)

위의 r변수를 실행하여 제대로 응답이 오는지 확인해주자. 응답코드 200이 나오면 정상이다.

In [8]:
r

<Response [200]>

url의 api를 통해 불려들어오는 json 데이터 포맷을 읽어오기 위해 r_json변수에 r.json()함수를 담는다

In [9]:
r_json = r.json()

위의 변수의 데이터가 제대로 응답하는지 알아보기 위해 '정상'이라는 메시지로 표기해주자

In [10]:
r_json['message']

'정상'

위의 변수를 데이터프레임으로 변환하는 소스코드를 입력하고 일부를 실행해서 확인해보자.

In [11]:
df = pd.DataFrame([r_json][0]['list'])

In [12]:
df

,rcept_no,reprt_code,bsns_year,corp_code,sj_div,sj_nm,account_id,account_nm,account_detail,thstrm_nm,thstrm_amount,frmtrm_nm,frmtrm_amount,bfefrmtrm_nm,bfefrmtrm_amount,ord,currency,thstrm_add_amount
0,20230324000011,11011,2022,00483735,BS,재무상태표,ifrs-full_CurrentAssets,유동자산,-,제 21 기,36478243625,제 20 기,70529362628,제 19 기,35281659914,1,KRW,NaN
1,20230324000011,11011,2022,00483735,BS,재무상태표,ifrs-full_CashAndCashEquivalents,현금및현금성자산,-,제 21 기,4460348176,제 20 기,22100156627,제 19 기,6467809011,2,KRW,NaN
2,20230324000011,11011,2022,00483735,BS,재무상태표,dart_ShortTermDepositsNotClassifiedAsCashEquiv...,단기금융상품,-,제 21 기,16320964,제 20 기,4043112750,제 19 기,4072086604,3,KRW,NaN
3,20230324000011,11011,2022,00483735,BS,재무상태표,ifrs-full_TradeAndOtherCurrentReceivables,매출채권및기타채권,-,제 21 기,12350676804,제 20 기,22492500453,제 19 기,10751878888,4,KRW,NaN
4,20230324000011,11011,2022,00483735,BS,재무상태표,ifrs-full_Inventories,재고자산,-,제 21 기,4140604587,제 20 기,3262613088,제 19 기,8074495197,5,KRW,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270,20230324000011,11011,2022,00483735,SCE,자본변동표,ifrs-full_Equity,기말자본,자본 [member]|지배기업의 소유주에게 귀속되는 자본 [member]|기타불입자...,제 21 기,2458935132,제 20 기,413123544,제 19 기,280886064,12,KRW,NaN
271,20230324000011,11011,2022,00483735,SCE,자본변동표,ifrs-full_Equity,기말자본,자본 [member]|지배기업의 소유주에게 귀속되는 자본 [member]|기타불입자...,제 21 기,75633671151,제 20 기,74893257404,제 19 기,48917565181,12,KRW,NaN
272,20230324000011,11011,2022,00483735,SCE,자본변동표,ifrs-full_Equity,기말자본,자본 [member]|지배기업의 소유주에게 귀속되는 자본 [member]|기타자본구...,제 21 기,5737031033,제 20 기,4695663839,제 19 기,2411203828,12,KRW,NaN
273,20230324000011,11011,2022,00483735,SCE,자본변동표,ifrs-full_Equity,기말자본,자본 [member]|지배기업의 소유주에게 귀속되는 자본 [member]|이익잉여금...,제 21 기,-106957951336,제 20 기,-91114460620,제 19 기,-50812013963,12,KRW,NaN


In [3]:
# ETL 방식을 활용해보자.
# ETL은 추출(Extract), 변환(Transform), 로드(Load)를 말한다.
# 조직에서 여러 시스템의 데이터를 단일 데이터베이스, 데이터 저장소, 데이터 웨어하우스 또는 데이터 레이크에 결합하기 위해 일반적으로 허용되는 방법이다.

여기에서는 DART 기업의 전체 재무제표를 불러오기 전에 일단 DART내의 일부분의 데이터를 불러오는 실습을 해보자.
이번에는 2022년 (분기, 반기, 3분기, 사업) 보고서 연결재무제표를 불러올 것이다. 따라서 다음과 같이 변수를 만들어 리스트 값을 담아주자.

In [14]:
bsns_year_list = ['2022']
reprt_code_list = ['11011','11012','11013','11015']
fs_div_val_list = ['CFS']

사업연도와 연결재무제표는 항목이 1개이므로 놔두고, 나머지 보고서 형식과 기업 고유번호만 반복문을 활용하여 담아주어야 하므로 val변수를 만들어 값을 넣어주자.

In [15]:
reprt_code_val = '1101521'
corp_code_val = '02168546'

앞서 위에서 준비를 했던 코드를 활용해보자.

In [16]:
url = 'https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json?'
params = f'crtfc_key={api_key}&corp_code={corp_code_val}&bsns_year={bsns_year_val}&reprt_code={reprt_code_val}&fs_div={fs_div_val}'

api_url = url + params

r = requests.get(api_url)
r_json = r.json()

In [44]:
api_url

'https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json?crtfc_key=801b9fc87e380dfc40a19a593d6fd762bc681388&corp_code=02168546&bsns_year=2022&reprt_code=1101521&fs_div=CFS'

In [46]:
r_json

{'status': '013', 'message': '조회된 데이타가 없습니다.'}

try-except 구문을 활용해보자.(DART에서 값이 누락되는 기업이 많으므로 예외처리를 해줘야한다.)

In [ ]:
try:

except:


In [ ]:
bsns_year_list = ['2022']
reprt_code_list = ['11011','11012','11013','11015']
fs_div_val_list = ['CFS']

In [19]:
bsns_year_val = '2022'
fs_div_val = 'CFS'

해당 데이터 값이 있을 경우와 없을 경우로 나누어 값이 있는 경우만 데이터를 디렉토리에 값을 담아주자.

In [17]:
if not os.path.exists(f'fnlttSinglAcntAll'):
    os.makedirs('fnlttSinglAcntAll')

위에서 준비했던 코드들을 이용하여 실제로 데이터를 수집하는 소스코드를 짜면 다음과 같다. for구문을 활용하여 각각 다양한 데이터들을 반복해서 담아주고, try-except 구문으로 누락 데이터값을 예외처리 해주자. 그리고 여기에선 if not- else 구문으로 데이터가 있을 경우와 없을 경우 항목을 나누어두었다. log로 불러온 데이터값을 데이터프레임으로 변환하여 lof_df.csv에 최종적으로 저장하도록 코드를 짰다.

In [ ]:
# total = pd.DataFrame()
for corp_code_val in corp_info_stock_code['corp_code']:  # corp code
    for reprt_code_val in reprt_code_list: # reprt code (11011, 11012, 11013, 11014)
        print(f'{corp_code_val}_{bsns_year_val}_{reprt_code_val}_{fs_div_val}')
        
        url = 'https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json?'
        params = f'crtfc_key={api_key}&corp_code={corp_code_val}&bsns_year={bsns_year_val}&reprt_code={reprt_code_val}&fs_div={fs_div_val}'
    
        try:
    
            api_url = url + params
   
            r = requests.get(api_url)
            r_json = r.json()
            sleep(0.7)
            
            df = pd.DataFrame([r_json][0]['list'])
            
            df.to_csv(f'fnlttSinglAcntAll/{corp_code_val}_{bsns_year_val}_{reprt_code_val}_{fs_div_val}.csv', index = False)
            
            # log
            now = datetime.now()
            log_df = pd.DataFrame({
                'corp_code':corp_code_val,
                'bsns_year':bsns_year_val,
                'reprt_code':reprt_code_val,
                'fs_div':fs_div_val,
                'time': now.strftime('%Y-%m-%d %H:%M:%S'),
                'status':r_json['message']            
            },index = [0])
            
            if not os.path.exists(f'log_fnlttSinglAcntAll.csv'):
                log_df.to_csv(f'log_fnlttSinglAcntAll.csv', index = False, mode = 'w')
            else:
                log_df.to_csv(f'log_fnlttSinglAcntAll.csv', index = False, mode = 'a', header = False)    
            print('success')
            
         
        except:
            # log
            now = datetime.now()
            log_df = pd.DataFrame({
                'corp_code':corp_code_val,
                'bsns_year':bsns_year_val,
                'reprt_code':reprt_code_val,
                'fs_div':fs_div_val,
                'time': now.strftime('%Y-%m-%d %H:%M:%S'),
                'status':r_json['message']            
            },index = [0])
            
            if not os.path.exists(f'log_fnlttSinglAcntAll.csv'):
                log_df.to_csv(f'log_fnlttSinglAcntAll.csv', index = False, mode = 'w')
            else:
                log_df.to_csv(f'log_fnlttSinglAcntAll.csv', index = False, mode = 'a', header = False)   
            print('failed')
            
#     df = pd.DataFrame([r_json][0]['list'])    
#     total = pd.concat([total,df])

00260985_2022_11011_CFS
failed
00260985_2022_11012_CFS
failed
00260985_2022_11013_CFS
failed
00260985_2022_11015_CFS
failed
00264529_2022_11011_CFS
failed
00264529_2022_11012_CFS
failed
00264529_2022_11013_CFS
failed
00264529_2022_11015_CFS
failed
00358545_2022_11011_CFS
failed
00358545_2022_11012_CFS
failed
00358545_2022_11013_CFS
failed
00358545_2022_11015_CFS
failed
00231567_2022_11011_CFS
failed
00231567_2022_11012_CFS
failed
00231567_2022_11013_CFS
failed
00231567_2022_11015_CFS
failed
00247939_2022_11011_CFS
failed
00247939_2022_11012_CFS
failed
00247939_2022_11013_CFS
failed
00247939_2022_11015_CFS
failed
00359614_2022_11011_CFS
failed
00359614_2022_11012_CFS
failed
00359614_2022_11013_CFS
failed
00359614_2022_11015_CFS
failed
00153551_2022_11011_CFS
failed
00153551_2022_11012_CFS
failed
00153551_2022_11013_CFS
failed
00153551_2022_11015_CFS
failed
00344746_2022_11011_CFS
failed
00344746_2022_11012_CFS
failed
00344746_2022_11013_CFS
failed
00344746_2022_11015_CFS
failed
00261188

밑의 코드는 해당 데이터가 존재하는지 여부를 보기 위한 코드다.

In [49]:
os.path.exists('company_info.csv')

True